In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('HamSpam').getOrCreate()

#**Part 1 - Data Preprocessing**

In [ ]:
df = spark.read.csv("/FileStore/tables/smsspamcollection.tsv",inferSchema=True,sep='\t',header=True)
df.show(5)

+-----+--------------------+------+-----+
label| message|length|punct|
+-----+--------------------+------+-----+
 ham|Go until jurong p...| 111| 9|
 ham|Ok lar... Joking ...| 29| 6|
 spam|Free entry in 2 a...| 155| 6|
 ham|U dun say so earl...| 49| 6|
 ham|Nah I don't think...| 61| 2|
+-----+--------------------+------+-----+
only showing top 5 rows

In [ ]:
df = df.withColumnRenamed('label','label_cat')
df.columns

Out[42]: ['label_cat', 'message', 'length', 'punct']

**1.1 Cleaning Null Values**

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show() #the survival label is mostly null, so lets remove it anyway since this is unsupervised. Also remove _c9 column which is entirely null

+---------+-------+------+-----+
label_cat|message|length|punct|
+---------+-------+------+-----+
 0| 0| 2| 2|
+---------+-------+------+-----+

**1.2 TFIDF**

In [ ]:
tokenize = Tokenizer(inputCol="message", outputCol="token_message")
stopwords = StopWordsRemover(inputCol='token_message',outputCol='stop_tokens')
count_vectorize = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
hamspam_cat = StringIndexer(inputCol='label_cat',outputCol='label')

In [ ]:
features = VectorAssembler(inputCols=['tf_idf'],outputCol='features')

**1.3 Linear SVC Model**


In [ ]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

# **Part 2 - Creating Pipeline**


In [ ]:
pipe = Pipeline(stages=[hamspam_cat,tokenize,stopwords,count_vectorize,idf,features])

In [ ]:
pipeline_model = pipe.fit(df)

In [ ]:
pipeline_transform = pipeline_model.transform(df)

#**Part 3 - Training and Evaluating Model**


In [ ]:
df_final = pipeline_transform.select(['label','features'])
df_final.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13494,[7,11,31,6...|
 0.0|(13494,[0,24,296,...|
 1.0|(13494,[2,13,19,2...|
 0.0|(13494,[0,69,79,1...|
 0.0|(13494,[36,134,31...|
+-----+--------------------+
only showing top 5 rows

**3.1 Train Test Split**


In [ ]:
train, test = df_final.randomSplit([0.8,0.2]) #train will have 80%, test will have 20% of data

**3.2 Training on our Linear SVC**


In [ ]:
pred = lsvc.fit(train)

In [ ]:
pred_test = pred.transform(test)
pred_test.show(5)

+-----+--------------------+--------------------+----------+
label| features| rawPrediction|prediction|
+-----+--------------------+--------------------+----------+
 0.0|(13494,[0,1,2,13,...|[1.74095054791060...| 0.0|
 0.0|(13494,[0,1,5,20,...|[2.32779765017973...| 0.0|
 0.0|(13494,[0,1,7,15,...|[2.89125419219622...| 0.0|
 0.0|(13494,[0,1,18,20...|[2.58326156346535...| 0.0|
 0.0|(13494,[0,1,876],...|[0.98416539171552...| 0.0|
+-----+--------------------+--------------------+----------+
only showing top 5 rows

**Evaluating Results**

In [ ]:
evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(pred_test)
print("Accuracy of model at predicting spam was: {}".format(accuracy))

Accuracy of model at predicting spam was: 0.9722641247263415